# Understanding input pipeline optimizations

This notebooks is based on the tutorial [Better performance with the tf.data API](https://www.tensorflow.org/guide/data_performance) from TensorFlow's documentation.

Here we are going to use a generator to simulate opening and reading a file to create a schematic timeline of the input pipeline. The time spent in opening the data, reading it, mapping and training will be simulated with `time.sleep()`. We are going to generate a trace event json file and visualize it with Chrome's tracing utility (chrome://tracing/)

The idea now is to loop over the dataset simulating the training and apply `map` with and wihout `num_parallel_calls`, add `cache` and `prefetch` transformations and see what happens. The 'training' produces the file `tfdata.json` that we need to copy to the computer where we have Chrome.

More info:
 * [Trace Event Format](https://docs.google.com/document/d/1CvAClvFfyA5R-PhYUmn5OOQtYMH4h6I0nSsKchNAySU/preview)

In [1]:
import json
import time
import tensorflow as tf

In [2]:
def generator(num_samples):
    # Opening the file
    open_enter = time.perf_counter()
    time.sleep(0.03)
    open_elapsed = time.perf_counter() - open_enter

    for sample_idx in range(num_samples):
        # Reading data (line, record) from the file
        read_enter = time.perf_counter()
        time.sleep(0.02)
        read_elapsed = time.perf_counter() - read_enter

        yield (
            [("Open",), ("Read",)],
            [(open_enter, open_elapsed), (read_enter, read_elapsed)],
        )
        # Negative values will be filtered out
        open_enter, open_elapsed = -1., -1.

In [3]:
# The map function needs to be wrapped using `tf.py_function`
# to prevent auto-graph from compiling it
# If the function is compiled, then the starting time of the
# map operation will be allways the same
def map_decorator(func):
    def wrapper(steps, times):
        return tf.py_function(
            func,
            inp=(steps, times),
            Tout=(steps.dtype, times.dtype)
        )
    return wrapper

In [4]:
@map_decorator
def mapping(*sample):
    map_start = time.perf_counter()
    time.sleep(0.03)
    map_elapsed = time.perf_counter() - map_start
    return (
        [sample[0][0], sample[0][1], ('Map',)],
        [(sample[1][0][0], sample[1][0][1]),
         (sample[1][1][0], sample[1][1][1]),
         (map_start, map_elapsed)]
    )

In [24]:
num_samples = 20
dataset = tf.data.Dataset.from_generator(generator,
                                         output_types=(tf.dtypes.string, tf.dtypes.float64),
                                         output_shapes=((2, 1), (2, 2)),
                                         args=(num_samples,)
                                        )
# dataset = dataset.map(mapping, num_parallel_calls=12)
# dataset = dataset.cache()
# dataset = dataset.prefetch(10)   #tf.data.experimental.AUTOTUNE)

In [25]:
num_epochs = 2

us = 1e6
trace_events = {'traceEvents': []}
for epoch in range(num_epochs):
    for ops, times in dataset:
        for op, t in zip(ops, times):
            if t.numpy()[0] <= 0:
                continue

            name = f'{op[0]}'[2:-1]  # from tf it comes as b'open'
            ts = t.numpy()[0]
            dur = t.numpy()[1]

            trace_events['traceEvents'].append({"pid": 1,
                                                "tid": name,
                                                "ts": int(ts * us),
                                                "dur": int(dur * us),
                                                "ph":"X",
                                                "name": name})
        train_start = time.perf_counter()
        train_dur = 0.01
        time.sleep(train_dur)
        trace_events['traceEvents'].append({"pid": 1,
                                            "tid": 'Train',
                                            "ts": int(train_start * us),
                                            "dur": int(train_dur * us),
                                            "ph":"X",
                                            "name": 'Train'})

with open('tfdata.json', 'w') as f:
    json.dump(trace_events, f, indent=4)